In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split  # Untuk membagi data menjadi latih dan uji
from sklearn.preprocessing import StandardScaler  # Untuk normalisasi data
from sklearn.metrics import accuracy_score  # Untuk menghitung akurasi
import matplotlib.pyplot as plt  # Untuk visualisasi hasil
from torch.utils.data import DataLoader, TensorDataset  # Untuk membuat DataLoader
from fpdf import FPDF  # Untuk membuat laporan PDF

In [2]:
# 1. Data Preprocessing
df = pd.read_csv('winequality-white.csv', delimiter=';')

# Pisahkan fitur (X) dan target (y)
X = df.drop('quality', axis=1).values  # Semua kolom selain 'quality' sebagai fitur
y = df['quality'].values  # Kolom 'quality' sebagai target

# Normalisasi data fitur agar nilai berada dalam rentang yang sama
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)  # Fit dan transform data fitur

# Membagi data menjadi set latih (80%) dan uji (20%)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Mengubah data menjadi tensor PyTorch agar bisa digunakan dalam pelatihan
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

# Membuat dataset Tensor untuk DataLoader
train_data = TensorDataset(X_train_tensor, y_train_tensor)
test_data = TensorDataset(X_test_tensor, y_test_tensor)

# Membuat DataLoader untuk batch training dan testing
train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
test_loader = DataLoader(test_data, batch_size=64, shuffle=False)


In [3]:
# 2. Model Definition: Vanilla MLP
class MLP(nn.Module):
    def __init__(self, input_dim, hidden_layers, hidden_neurons, activation_fn):
        super(MLP, self).__init__()
        layers = []  # List untuk menyimpan layer-layer

        prev_layer = input_dim  # Dimensi input adalah jumlah fitur

        # Menambahkan hidden layers sesuai jumlah yang ditentukan
        for _ in range(hidden_layers):
            layers.append(nn.Linear(prev_layer, hidden_neurons))  # Menambahkan layer Linear
            if activation_fn == 'relu':  # Jika fungsi aktivasi ReLU
                layers.append(nn.ReLU())  # Menambahkan ReLU
            elif activation_fn == 'sigmoid':  # Jika fungsi aktivasi Sigmoid
                layers.append(nn.Sigmoid())  # Menambahkan Sigmoid
            elif activation_fn == 'tanh':  # Jika fungsi aktivasi Tanh
                layers.append(nn.Tanh())  # Menambahkan Tanh
            elif activation_fn == 'softmax':  # Jika fungsi aktivasi Softmax
                layers.append(nn.Softmax(dim=1))  # Menambahkan Softmax di layer akhir
            prev_layer = hidden_neurons  # Setel prev_layer ke jumlah neuron di layer sebelumnya
        
        # Menambahkan layer output, di sini outputnya adalah 10 kelas
        layers.append(nn.Linear(prev_layer, 10))  # 10 kelas (quality wine)

        # Menyusun layer-layer ke dalam model
        self.model = nn.Sequential(*layers)

    # Mendefinisikan fungsi forward untuk propagasi data lewat jaringan
    def forward(self, x):
        return self.model(x)  # Mengembalikan hasil dari model


In [4]:
# 3. Training & Evaluation Function (Perubahan di sini)
def train_model(model, criterion, optimizer, epochs, train_loader, test_loader):
    train_losses = []  # Menyimpan loss selama pelatihan
    test_accuracies = []  # Menyimpan akurasi setiap epoch

    for epoch in range(epochs):
        model.train()  # Menyiapkan model untuk pelatihan
        running_loss = 0.0  # Variabel untuk menghitung loss selama satu epoch

        # Loop untuk setiap batch di train_loader
        for inputs, labels in train_loader:
            # Forward pass: menghitung output dari model
            outputs = model(inputs)
            loss = criterion(outputs, labels)  # Menghitung loss

            # Backward pass dan optimasi: memperbarui parameter model
            optimizer.zero_grad()  # Mengatur gradient ke 0 sebelum backward pass
            loss.backward()  # Menghitung gradien
            optimizer.step()  # Mengupdate parameter model

            running_loss += loss.item()  # Menambah loss per batch ke total loss
        
        # Menyimpan rata-rata loss untuk epoch ini
        train_losses.append(running_loss / len(train_loader))

        # Evaluasi model pada data uji setelah tiap epoch
        model.eval()  # Menyiapkan model untuk evaluasi
        correct = 0  # Variabel untuk menghitung jumlah prediksi yang benar
        total = 0  # Variabel untuk menghitung total data

        with torch.no_grad():  # Menonaktifkan gradient calculation
            for inputs, labels in test_loader:
                outputs = model(inputs)  # Menghitung output model
                _, predicted = torch.max(outputs, 1)  # Mengambil kelas dengan probabilitas tertinggi
                total += labels.size(0)  # Menambahkan jumlah data
                correct += (predicted == labels).sum().item()  # Menghitung prediksi yang benar
        
        accuracy = 100 * correct / total  # Menghitung akurasi
        test_accuracies.append(accuracy)  # Menyimpan akurasi per epoch
    
    # Menampilkan hanya akurasi terbaik akhir epoch
    print(f"Final Accuracy: {test_accuracies[-1]}%")
    
    return train_losses, test_accuracies  # Mengembalikan loss dan akurasi untuk analisis


In [5]:
# 4. Hyperparameter Tuning
hidden_layers_options = [1, 2, 3]  # Jumlah hidden layers yang akan diuji
hidden_neurons_options = [4, 8, 16, 32, 64]  # Jumlah neuron di setiap hidden layer
activation_functions = ['relu', 'sigmoid', 'tanh', 'softmax']  # Fungsi aktivasi yang akan diuji
epochs_options = [25, 50, 100]  # Jumlah epoch yang akan diuji
learning_rates = [0.01, 0.001, 0.0001]  # Learning rates yang akan diuji
batch_sizes = [32, 64, 128]  # Ukuran batch yang akan diuji




best_accuracy = 0  # Inisialisasi akurasi terbaik
best_params = {}  # Menyimpan hyperparameter terbaik

activation_results = {
    'relu': None,
    'sigmoid': None,
    'tanh': None,
    'softmax': None
}

# Loop untuk mencoba semua kombinasi hyperparameter
for hidden_layers in hidden_layers_options:
    for hidden_neurons in hidden_neurons_options:
        for activation_fn in activation_functions:
            for epochs in epochs_options:
                for lr in learning_rates:
                    for batch_size in batch_sizes:
                        print(f"Training with {hidden_layers} hidden layers, {hidden_neurons} neurons, {activation_fn} activation, {epochs} epochs, {lr} learning rate, {batch_size} batch size")
                        
                        # Membuat model dan optimizer
                        model = MLP(X_train.shape[1], hidden_layers, hidden_neurons, activation_fn)
                        criterion = nn.CrossEntropyLoss()  # Fungsi loss untuk klasifikasi
                        optimizer = optim.Adam(model.parameters(), lr=lr)  # Optimizer Adam dengan learning rate yang dipilih
                        
                        # Melatih model
                        train_losses, test_accuracies = train_model(model, criterion, optimizer, epochs, train_loader, test_loader)
                        
                        # Evaluasi akurasi model pada akhir epoch
                        final_accuracy = test_accuracies[-1]
                        if final_accuracy > best_accuracy:  # Jika akurasi lebih baik dari sebelumnya
                            best_accuracy = final_accuracy  # Simpan akurasi terbaik
                            best_params = {
                                'hidden_layers': hidden_layers,
                                'hidden_neurons': hidden_neurons,
                                'activation_fn': activation_fn,
                                'epochs': epochs,
                                'lr': lr,
                                'batch_size': batch_size
                            }

                        # Menyimpan parameter terbaik per fungsi aktivasi
                        if final_accuracy > (activation_results[activation_fn]['best_accuracy'] if activation_results[activation_fn] else 0):
                            activation_results[activation_fn] = {
                                'best_params': best_params,
                                'best_accuracy': final_accuracy
                            }

print("Best Hyperparameters:", best_params)  # Menampilkan hyperparameter terbaik
print("Best Accuracy:", best_accuracy)  # Menampilkan akurasi terbaik


Training with 1 hidden layers, 4 neurons, relu activation, 25 epochs, 0.01 learning rate, 32 batch size
Final Accuracy: 52.55102040816327%
Training with 1 hidden layers, 4 neurons, relu activation, 25 epochs, 0.01 learning rate, 64 batch size
Final Accuracy: 51.53061224489796%
Training with 1 hidden layers, 4 neurons, relu activation, 25 epochs, 0.01 learning rate, 128 batch size
Final Accuracy: 52.857142857142854%
Training with 1 hidden layers, 4 neurons, relu activation, 25 epochs, 0.001 learning rate, 32 batch size
Final Accuracy: 51.83673469387755%
Training with 1 hidden layers, 4 neurons, relu activation, 25 epochs, 0.001 learning rate, 64 batch size
Final Accuracy: 51.326530612244895%
Training with 1 hidden layers, 4 neurons, relu activation, 25 epochs, 0.001 learning rate, 128 batch size
Final Accuracy: 52.6530612244898%
Training with 1 hidden layers, 4 neurons, relu activation, 25 epochs, 0.0001 learning rate, 32 batch size
Final Accuracy: 25.918367346938776%
Training with 1 hi

In [6]:
# 5. Menyusun Kesimpulan Berdasarkan Fungsi Aktivasi
print("\nBest Hyperparameters for each Activation Function:")
for activation_fn, result in activation_results.items():
    print(f"\nActivation Function: {activation_fn.capitalize()}")
    print(f"Best Hyperparameters: {result['best_params']}")
    print(f"Best Accuracy: {result['best_accuracy']}")



Best Hyperparameters for each Activation Function:

Activation Function: Relu
Best Hyperparameters: {'hidden_layers': 2, 'hidden_neurons': 64, 'activation_fn': 'tanh', 'epochs': 100, 'lr': 0.01, 'batch_size': 32}
Best Accuracy: 62.6530612244898

Activation Function: Sigmoid
Best Hyperparameters: {'hidden_layers': 3, 'hidden_neurons': 64, 'activation_fn': 'sigmoid', 'epochs': 100, 'lr': 0.01, 'batch_size': 64}
Best Accuracy: 64.08163265306122

Activation Function: Tanh
Best Hyperparameters: {'hidden_layers': 3, 'hidden_neurons': 64, 'activation_fn': 'tanh', 'epochs': 100, 'lr': 0.01, 'batch_size': 128}
Best Accuracy: 67.55102040816327

Activation Function: Softmax
Best Hyperparameters: {'hidden_layers': 2, 'hidden_neurons': 64, 'activation_fn': 'tanh', 'epochs': 100, 'lr': 0.01, 'batch_size': 32}
Best Accuracy: 58.673469387755105
